In [83]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
import keras

In [84]:
def text_process(mess):
    lst = [x for x in mess if x not in string.punctuation + '0123456789' ]
    mess = ''.join(lst)
    return [word for word in mess.split() if word.lower() not in stopwords.words('english')]


In [85]:
messages = pd.read_csv('E:/Study/NLTK/Spam-NotSpam/SMSSpamCollection',sep='\t',names = ['label','message'])
messages['length'] = (messages['message']).apply(len)

In [86]:
messages['label'] = messages['label'].map({'ham':0,'spam':1})
test_data = messages['label']
test_data.head()

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64

In [87]:
tokenizer = Tokenizer()

In [88]:
tokenizer.fit_on_texts(messages['message'])
sequences_train = tokenizer.texts_to_sequences(messages['message'])

In [89]:
sequences_train

[[49,
  471,
  4436,
  842,
  755,
  658,
  64,
  8,
  1327,
  88,
  123,
  351,
  1328,
  148,
  2997,
  1329,
  67,
  58,
  4437,
  144],
 [46, 336, 1499, 472, 6, 1940],
 [47,
  489,
  8,
  19,
  4,
  797,
  901,
  2,
  176,
  1941,
  1105,
  659,
  1942,
  2331,
  261,
  2332,
  71,
  1941,
  2,
  1943,
  2,
  337,
  489,
  555,
  960,
  73,
  391,
  174,
  660,
  392,
  2998],
 [6, 248, 150, 23, 382, 2999, 6, 139, 154, 57, 150],
 [1024, 1, 98, 108, 69, 490, 2, 961, 69, 1944, 221, 112, 473],
 [798,
  129,
  67,
  1690,
  145,
  109,
  158,
  1945,
  21,
  7,
  38,
  338,
  89,
  902,
  55,
  116,
  414,
  3,
  44,
  12,
  14,
  86,
  1946,
  46,
  365,
  960,
  4438,
  2,
  68,
  323,
  232,
  2,
  3000],
 [210, 11, 633, 9, 25, 55, 2, 383, 36, 10, 110, 718, 10, 55, 4439, 4440],
 [72,
  235,
  13,
  1204,
  2333,
  2334,
  1947,
  2335,
  2336,
  2337,
  799,
  118,
  109,
  609,
  72,
  13,
  1025,
  12,
  51,
  1691,
  843,
  393,
  2,
  1106,
  13,
  249,
  1025],
 [719,
  72,
  4

In [90]:
word_index = tokenizer.word_index

In [91]:
print('Found %s unique tokens.' % len(word_index))

Found 9009 unique tokens.


In [92]:
max_input_lenght = max([len(x) for x in sequences_train])

In [93]:
max_input_lenght

189

In [94]:
train_data = keras.preprocessing.sequence.pad_sequences(sequences_train,
                                                        padding='post',
                                                        maxlen=max_input_lenght)


In [95]:
train_data[2]

array([  47,  489,    8,   19,    4,  797,  901,    2,  176, 1941, 1105,
        659, 1942, 2331,  261, 2332,   71, 1941,    2, 1943,    2,  337,
        489,  555,  960,   73,  391,  174,  660,  392, 2998,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [96]:
vocab_size = 10000
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [97]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [98]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [99]:
model.fit(train_data,
                    test_data,
                    epochs=40,
                    batch_size=512,
                    verbose=1)

Epoch 1/40
5572/5572 [==============================] - 6s 1ms/step - loss: 0.6991 - acc: 0.3693
Epoch 2/40
5572/5572 [==============================] - 3s 518us/step - loss: 0.6730 - acc: 0.8598
Epoch 3/40
5572/5572 [==============================] - 3s 593us/step - loss: 0.6402 - acc: 0.8659
Epoch 4/40
5572/5572 [==============================] - 3s 603us/step - loss: 0.6014 - acc: 0.8659
Epoch 5/40
5572/5572 [==============================] - 3s 568us/step - loss: 0.5602 - acc: 0.8659
Epoch 6/40
5572/5572 [==============================] - 4s 702us/step - loss: 0.5186 - acc: 0.8659
Epoch 7/40
5572/5572 [==============================] - 3s 578us/step - loss: 0.4808 - acc: 0.8659
Epoch 8/40
5572/5572 [==============================] - 3s 499us/step - loss: 0.4474 - acc: 0.8659
Epoch 9/40
5572/5572 [==============================] - 3s 453us/step - loss: 0.4211 - acc: 0.8659
Epoch 10/40
5572/5572 [==============================] - 4s 704us/step - loss: 0.3984 - acc: 0.8659
Epoch 11/40

In [66]:
results = model.evaluate(train_data[:100], test_data[:100])

100/100 [==============================] - 0s 2ms/step


In [82]:
model.predict(np.array([  47,  489,    8,   19,    4,  797,  901,    2,  176, 1941, 1105,
        659, 1942, 2331,  261, 2332,   71, 1941,    2, 1943,    2,  337,
        489,  555,  960,   73,  391,  174,  660,  392, 2998,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0]))[0][0]

0.97918975